<img src="https://www3.um.edu.uy/logoum.jpg" width=300>
<h1 align="center">Thesis - IMF nDCG</h1> 
<h2 align="center">Alejo Paullier</h2> 

In [1]:
from lenskit import batch, topn, util
from lenskit import crossfold as xf
from lenskit.algorithms import Recommender, als
from lenskit.algorithms.als import ImplicitMF
from lenskit import topn
import pandas as pd
import numpy as np

ratings = pd.read_csv('D:\\Escritorio\\UM\\Tesis\\ML 1M\\ratings.dat', sep='::',engine='python',
                      names=['user', 'item', 'rating', 'timestamp'])

def eval(aname, algo, train, test):
    fittable = util.clone(algo) # Object cloning means to create an exact copy of the original object.
    fittable = Recommender.adapt(fittable) 
    fittable.fit(train) # Entrenamos el algoritmo con el training dataset
    users = test.user.unique() # Devuelve un array con los users unicos en el testing dataset
    # now we run the recommender
    recs = batch.recommend(fittable, users, 100)
    # add the algorithm name for analyzability
    recs['Algorithm'] = aname
    return recs

In [2]:
def model_trainer(features,reg,weight):
    all_recs = []
    test_data = []
    
    features=int(features)
    imf = ImplicitMF(features=features, iterations=100, reg=reg, weight=weight) # define algorithm# define algorithm
    
    for train, test in xf.partition_users(ratings[['user', 'item', 'rating']], 5, xf.SampleFrac(0.2)):
        test_data.append(test) # save testing data
        all_recs.append(eval('IMF', imf, train, test))
    
    all_recs = pd.concat(all_recs, ignore_index=True)
    test_data = pd.concat(test_data, ignore_index=True)
    rla = topn.RecListAnalysis()
    rla.add_metric(topn.ndcg)
    results = rla.compute(all_recs, test_data)
                          
    return results["ndcg"].mean()

In [ ]:
from hyperopt import fmin, tpe, hp, STATUS_OK
array=[]
def objective(params):
    features = params['features']
    reg = params['reg']
    weight=params['weight']
    metric = model_trainer(features,reg,weight)
    array.append([features,reg,weight,metric])
    return {'loss': -metric, 'status': STATUS_OK}

space={'features': hp.uniform('features', 1, 100),
       'reg': hp.uniform('reg',0.01,0.4),
      'weight': hp.uniform('weight',0,100)}

best = fmin(objective, space, algo=tpe.suggest,max_evals=100)

In [4]:
array #666.3

[[80.28697316962632,
  0.32754720101389917,
  25.26656290818785,
  0.2906581655937013],
 [90.18779255884029,
  0.19974162610105692,
  83.77444678405361,
  0.2291780765806615],
 [62.44668796254922,
  0.09229042244354366,
  12.44613057902959,
  0.3405750280246988],
 [25.970866025845833,
  0.31901126902248705,
  48.364076024514524,
  0.23284982948862168],
 [70.90945000573895,
  0.16339892437589748,
  33.88492432125977,
  0.2691814595241564],
 [50.921521623729184,
  0.21788338676967678,
  72.02982225854248,
  0.21181505827788866],
 [46.18029524598902,
  0.1340437885087975,
  7.295328708551175,
  0.37522286426991913],
 [13.87740457054501,
  0.05431449503479194,
  15.188321001830118,
  0.31491068728409205],
 [68.89158592611193,
  0.3800424460256505,
  33.789738630080656,
  0.2669136780584352],
 [21.180702152061226,
  0.019128551219099975,
  86.94049013907113,
  0.18528011518446588],
 [92.66623816829103,
  0.37756927333004703,
  62.51245983766604,
  0.24107648116330962],
 [8.235282120650078,


In [4]:
array #414

[[78.01854899714316,
  0.08774233578023034,
  22.650159272615777,
  0.2961240342689049],
 [2.1141139145143395,
  0.2230049170065692,
  51.88973558768977,
  0.19329044622945302],
 [17.90094018272824,
  0.16400323140259737,
  11.710731011586983,
  0.3382456762455465],
 [93.04865026740286,
  0.3381416737077969,
  36.58194447314832,
  0.2701632817158909],
 [79.95782609888435,
  0.26061836828600593,
  23.56821797598574,
  0.293763112894899],
 [96.09402939224977,
  0.040439673597644644,
  85.19760258234793,
  0.23081383930119817],
 [68.78154319286428,
  0.3276423577936688,
  24.620786157378806,
  0.28748727119680545],
 [77.54374785376163,
  0.04261743315297192,
  95.77543964281276,
  0.21486724613187905],
 [49.82652924048225,
  0.19368918711780472,
  65.4336651476195,
  0.21583037680218334],
 [25.55925410796567, 0.2709333369143615, 48.3917079445517, 0.2352797103118711],
 [68.94950348395737,
  0.2077201505359261,
  81.17638340297547,
  0.2166291043830872],
 [8.181856018341188,
  0.14960783657

In [5]:
array = [[78.01854899714316,
  0.08774233578023034,
  22.650159272615777,
  0.2961240342689049],
 [2.1141139145143395,
  0.2230049170065692,
  51.88973558768977,
  0.19329044622945302],
 [17.90094018272824,
  0.16400323140259737,
  11.710731011586983,
  0.3382456762455465],
 [93.04865026740286,
  0.3381416737077969,
  36.58194447314832,
  0.2701632817158909],
 [79.95782609888435,
  0.26061836828600593,
  23.56821797598574,
  0.293763112894899],
 [96.09402939224977,
  0.040439673597644644,
  85.19760258234793,
  0.23081383930119817],
 [68.78154319286428,
  0.3276423577936688,
  24.620786157378806,
  0.28748727119680545],
 [77.54374785376163,
  0.04261743315297192,
  95.77543964281276,
  0.21486724613187905],
 [49.82652924048225,
  0.19368918711780472,
  65.4336651476195,
  0.21583037680218334],
 [25.55925410796567, 0.2709333369143615, 48.3917079445517, 0.2352797103118711],
 [68.94950348395737,
  0.2077201505359261,
  81.17638340297547,
  0.2166291043830872],
 [8.181856018341188,
  0.14960783657143992,
  72.62480453115539,
  0.19726373118415286],
 [69.67830916248487,
  0.3584070263804792,
  93.09217117380429,
  0.20855887608733345],
 [49.155155599216734,
  0.3839616674413292,
  49.42496767325343,
  0.23618971705421604],
 [93.99137314765744,
  0.13771095924269894,
  42.87602711628426,
  0.260390996014326],
 [35.21006280447024,
  0.025500983355384595,
  45.094134975513846,
  0.24200230346329765],
 [77.1756566816792,
  0.12944596207132808,
  30.840609025553345,
  0.27707610874460087],
 [56.07724088774847,
  0.04157964609978465,
  10.362328909855034,
  0.3507274486825878],
 [98.53174029696117,
  0.07979368309362032,
  35.86305525922297,
  0.2725298643386989],
 [43.84876666854958,
  0.03421557813863642,
  43.5126505399425,
  0.24437558141028023],
 [19.26498440993459,
  0.09638163940208148,
  1.2550757204965866,
  0.4059922575150112],
 [60.74425763390746,
  0.08224513215014673,
  2.5533497381584263,
  0.4205074749918131],
 [34.99029470991099,
  0.09625287215599834,
  2.2723453340102995,
  0.4166787708712272],
 [36.17734541744584,
  0.10056466377343402,
  0.10461033970458278,
  0.4342827402710991],
 [59.67701342545874,
  0.0681593718971977,
  12.807703904293518,
  0.33541922627652787],
 [38.45499827869134,
  0.11618926250070735,
  1.4380632597043954,
  0.43132067409243074],
 [36.85830284877349,
  0.17256893005206841,
  16.194633362179047,
  0.3220711215277726],
 [27.138180728337932,
  0.11717355280082425,
  58.44959779613009,
  0.21839948182412303],
 [40.56624687137963,
  0.12099878613446233,
  5.225872597877263,
  0.39158583874922737],
 [26.121569601470693,
  0.23253668512965586,
  18.66134402819477,
  0.31578226329467834],
 [9.720974549436779,
  0.014631115567957001,
  8.71776971091019,
  0.3289519759015321],
 [16.120081564210512,
  0.16903810434617195,
  27.310755071572256,
  0.282642620822529],
 [2.2689503929157695,
  0.06113922312614495,
  19.791669534495014,
  0.20287296869150775],
 [44.99271706840239,
  0.2496121397813137,
  32.5873737089498,
  0.26715126081032875],
 [31.92665528876544,
  0.2813695225390651,
  57.32223754928211,
  0.22001119937714933],
 [55.46286007127651,
  0.18852511916321096,
  7.235344181083846,
  0.37155584521976925],
 [18.21197143001136,
  0.29117446906061784,
  15.151512386249355,
  0.32446320522312616],
 [29.518844331753236,
  0.10813893302544862,
  0.7642167193308741,
  0.42865022803399483],
 [23.265354151809007,
  0.15207585191239686,
  23.740343239844442,
  0.29368217425078497],
 [84.6658255592508,
  0.21724077545831805,
  39.007647279348944,
  0.26316409454156653],
 [10.31425132258444,
  0.18772316731704097,
  73.25212064516136,
  0.19886792071691953],
 [40.577326150708664,
  0.06255133375766242,
  21.29072063855524,
  0.30300227937966956],
 [1.2308024466548133,
  0.013286679441784494,
  27.64889667788766,
  0.2073641141944757],
 [62.9302050427451,
  0.3170546192043364,
  84.59786232260262,
  0.21112533155873697],
 [50.46760971779086, 0.1548134910379673, 56.50308027718627, 0.229409964099831],
 [46.03160592101435,
  0.23964946719393038,
  67.41812632061348,
  0.2140261984434995],
 [72.88229851364011,
  0.1392999880604471,
  90.57216172052254,
  0.21390769734154264],
 [88.69838620675338,
  0.05314051919128833,
  13.694831036286445,
  0.327493983691828],
 [53.593275571222215,
  0.21285016437346538,
  5.830318092043467,
  0.384497032079916],
 [64.77483814332481,
  0.10993601845418127,
  36.914446941885416,
  0.25937458252455414],
 [21.938290926362413,
  0.19851742037144537,
  98.10584052184697,
  0.17458567596922742],
 [39.37798096349132,
  0.07992278165155131,
  51.91300853389552,
  0.22945299698485125],
 [4.933060754198191,
  0.17679475044695292,
  31.354800590204835,
  0.22677961026737234],
 [31.831361627184943,
  0.13474578829261585,
  76.63249811968647,
  0.19935734674914696],
 [73.5400469433387,
  0.09608030441756371,
  40.3273661214156,
  0.25686985158898373],
 [13.456691424328213,
  0.03582792819161787,
  62.726817805846764,
  0.21546115774724658],
 [48.499229339615496,
  0.374909051510558,
  9.589917871484129,
  0.3606808494818099],
 [58.173519877085496,
  0.025832307308496394,
  0.16286267740008964,
  0.44301805268616345],
 [82.84341131825911,
  0.013912326009908077,
  27.89827276327161,
  0.2847366093755419],
 [66.48190598194289,
  0.04678380321578825,
  17.667111967625342,
  0.3107691439374212],
 [59.00304602359299,
  0.027597046591503905,
  10.89923326978718,
  0.3490495128799364],
 [72.63102440837498,
  0.06973141905150795,
  48.04606231662389,
  0.24696365211663815],
 [90.95513773441208,
  0.09299962573612819,
  4.265994624778244,
  0.39392264329584176],
 [53.0088741995722,
  0.02922312699370863,
  22.29326331294483,
  0.29623409355016445],
 [43.01748581920606,
  0.05188663460131117,
  52.6294890750725,
  0.2318102099106231],
 [36.823531294434666,
  0.12328726138907288,
  2.0714221075496035,
  0.42350871853202127],
 [32.8467843713445,
  0.10623973427650733,
  12.431695146407659,
  0.34283474809559555],
 [47.22449802974896,
  0.07549046413292296,
  7.404494650944677,
  0.37553133893476515],
 [28.11469753880965,
  0.15877169468193977,
  15.618253511634887,
  0.32673374544146594],
 [57.152266375761045,
  0.14235431999808432,
  3.87178091864562,
  0.4064615821558543],
 [23.04910204475586,
  0.08942851546258412,
  0.30750988132029344,
  0.4181616268663693],
 [42.93464402740416,
  0.12696623887601294,
  26.042728101075323,
  0.2854823743897549],
 [15.270201292079939,
  0.17929572930567444,
  19.40989950332199,
  0.3065671510170824],
 [61.77440450037241,
  0.11300896204460159,
  0.6655429742619596,
  0.44421063353088386],
 [75.74606911973035,
  0.22832148180576037,
  9.370310994278734,
  0.3519637231070175],
 [68.38366253316622,
  0.05995283907777475,
  34.14549523832453,
  0.2674015726470565],
 [79.74054747028134,
  0.019542610862785133,
  29.32610089280579,
  0.2818217474291273],
 [62.856695317861956, 0.1464542120463, 6.51885666747402, 0.3772624697353948],
 [51.21560796259469,
  0.16551261153885455,
  24.923439901320315,
  0.2895418782938151],
        [80.28697316962632,
  0.32754720101389917,
  25.26656290818785,
  0.2906581655937013],
 [90.18779255884029,
  0.19974162610105692,
  83.77444678405361,
  0.2291780765806615],
 [62.44668796254922,
  0.09229042244354366,
  12.44613057902959,
  0.3405750280246988],
 [25.970866025845833,
  0.31901126902248705,
  48.364076024514524,
  0.23284982948862168],
 [70.90945000573895,
  0.16339892437589748,
  33.88492432125977,
  0.2691814595241564],
 [50.921521623729184,
  0.21788338676967678,
  72.02982225854248,
  0.21181505827788866],
 [46.18029524598902,
  0.1340437885087975,
  7.295328708551175,
  0.37522286426991913],
 [13.87740457054501,
  0.05431449503479194,
  15.188321001830118,
  0.31491068728409205],
 [68.89158592611193,
  0.3800424460256505,
  33.789738630080656,
  0.2669136780584352],
 [21.180702152061226,
  0.019128551219099975,
  86.94049013907113,
  0.18528011518446588],
 [92.66623816829103,
  0.37756927333004703,
  62.51245983766604,
  0.24107648116330962],
 [8.235282120650078,
  0.11402289965000607,
  52.586901766614744,
  0.22559492145070897],
 [12.027020136409032,
  0.09982757195364389,
  58.46058526697641,
  0.2222129992199273],
 [41.2628137005819,
  0.1646657429273381,
  83.57843495918172,
  0.19703685958266173],
 [4.46967479707634,
  0.37172883555123004,
  96.49752318050892,
  0.18333928045231554],
 [51.54950295527345,
  0.3688121617539244,
  55.01573419241319,
  0.23010478931611664],
 [11.96496087637783, 0.2519619636935083, 32.571276163142, 0.26171610444497395],
 [49.75336544392465,
  0.010544989989570562,
  39.18948294636036,
  0.2567810759557532],
 [81.62907990503875,
  0.08673921767354043,
  61.38553296557466,
  0.2374705493886135],
 [19.992062306037933,
  0.1263704847836844,
  50.3146100774989,
  0.2343939808289037],
 [37.07603157174941,
  0.07387939800050797,
  1.129100382142596,
  0.4325760216115155]]


In [6]:
metric = pd.DataFrame(array)
metric.columns = ['features','reg','weight','nDCG']
metric.head(5)

,features,reg,weight,nDCG
0,78.018549,0.087742,22.650159,0.296124
1,2.114114,0.223005,51.889736,0.193290
2,17.900940,0.164003,11.710731,0.338246
3,93.048650,0.338142,36.581944,0.270163
4,79.957826,0.260618,23.568218,0.293763


In [7]:
metric.sort_values(by=['nDCG'],ascending=False).head(10)

,features,reg,weight,nDCG
73,61.774405,0.113009,0.665543,0.444211
57,58.173520,0.025832,0.162863,0.443018
23,36.177345,0.100565,0.104610,0.434283
99,37.076032,0.073879,1.129100,0.432576
25,38.454998,0.116189,1.438063,0.431321
37,29.518844,0.108139,0.764217,0.428650
65,36.823531,0.123287,2.071422,0.423509
21,60.744258,0.082245,2.553350,0.420507
70,23.049102,0.089429,0.307510,0.418162
22,34.990295,0.096253,2.272345,0.416679


In [8]:
metric.to_csv('metric_nDCG.csv', index=False)

In [3]:
metric = pd.read_csv('C:\\Users\\Alejo\\Tesis\\1M\\Implicit Matrix Factorization\\results\\metric_nDCG.csv')
metric

,features,reg,weight,nDCG
0,78.018549,0.087742,22.650159,0.296124
1,2.114114,0.223005,51.889736,0.193290
2,17.900940,0.164003,11.710731,0.338246
3,93.048650,0.338142,36.581944,0.270163
4,79.957826,0.260618,23.568218,0.293763
...,...,...,...,...
95,11.964961,0.251962,32.571276,0.261716
96,49.753365,0.010545,39.189483,0.256781
97,81.629080,0.086739,61.385533,0.237471
98,19.992062,0.126370,50.314610,0.234394


In [4]:
%matplotlib qt
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
colormap = cm.hot
x = metric['reg']
z = metric['features'] 
y = metric['weight']
c = metric['nDCG']

fig = plt.figure()

ax = fig.add_subplot(111, projection='3d')
img = ax.scatter(x, y, z, c=c, cmap=plt.jet())
ax.set_xlabel('reg')
ax.set_zlabel('features')
ax.set_ylabel('weight')
fig.colorbar(img)
plt.show()